<a href="https://colab.research.google.com/github/PaveenPaul/Machine-Learning-Projects-/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id unique if for news article
2. title: the title of news article
3. author of news article
4. text: the text of the articles; could be incomplete
5. labe: a label that markss whether the news article is real or fake
        1: Fake news
        0: real news

In [1]:
# Importing the dependencies

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Collection and preprocessing

In [9]:
# loading dataset into pandas dataframe
new_dataset = pd.read_csv("/content/drive/MyDrive/train.csv")

In [11]:
new_dataset.shape

(20800, 5)

In [13]:
# printing first 5 rows in the data set
new_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


1 - Real News

2 - Fake News

In [14]:
# counting the number of missing values in the dataset
new_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [15]:
# Replacing the null values with empty string
new_dataset = new_dataset.fillna("")

In [16]:
new_dataset.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [17]:
# seperating the data and the label

x =  new_dataset.drop(columns = "label", axis = 1)
y = new_dataset['label']

In [18]:
print(x)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [19]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Stemming:

Stemming is the Process of reducing a word to its root word (Removes all prefics and suffics)

example:
actor, actress, acting ----> root word is act .. so here act will be removed 

In [20]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [24]:
new_dataset['title'] = new_dataset['title'].apply(stemming)

In [28]:
print(new_dataset['title'].head())

0    hous dem aid even see comey letter jason chaff...
1      flynn hillari clinton big woman campu breitbart
2                                 truth might get fire
3             civilian kill singl us airstrik identifi
4    iranian woman jail fiction unpublish stori wom...
Name: title, dtype: object


In [33]:
x = new_dataset['title'].values
y = new_dataset['label'].values

In [32]:
print(x)

['hous dem aid even see comey letter jason chaffetz tweet'
 'flynn hillari clinton big woman campu breitbart' 'truth might get fire'
 ... 'maci said receiv takeov approach hudson bay new york time'
 'nato russia hold parallel exercis balkan' 'keep f aliv']


In [34]:
print(y)

[1 0 1 ... 0 1 1]


In [36]:
y.shape

(20800,)

In [37]:
# converting whats inthere in title i.e, text data into numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

In [39]:
print(x)

  (0, 13610)	0.3204429041871619
  (0, 11663)	0.28865071055944214
  (0, 7472)	0.3286263634991149
  (0, 6751)	0.37807503658407626
  (0, 6143)	0.24709678286281997
  (0, 4394)	0.2624700707018907
  (0, 3340)	0.3043365499413147
  (0, 2578)	0.2776884546129286
  (0, 2165)	0.4135903166037093
  (0, 243)	0.3038505731698672
  (1, 14597)	0.4029069338022535
  (1, 5974)	0.255243301322185
  (1, 4874)	0.49350006801559954
  (1, 2444)	0.25583271340004526
  (1, 1946)	0.5127916215485694
  (1, 1650)	0.21311459271319796
  (1, 1313)	0.3938922297906677
  (2, 13543)	0.520272613528009
  (2, 8285)	0.5879029381082882
  (2, 5278)	0.41416349493213955
  (2, 4770)	0.460603020443823
  (3, 13947)	0.2910988442735044
  (3, 12016)	0.4595551330231538
  (3, 7080)	0.3129596268768328
  (3, 6271)	0.48520717569885846
  :	:
  (20796, 14760)	0.12072790494289075
  (20796, 13246)	0.11914995894745603
  (20796, 11529)	0.4694159922014375
  (20796, 9959)	0.4804399319541416
  (20796, 9158)	0.4804399319541416
  (20796, 8881)	0.11669800295

Spliting the dataset to training and test data

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size = 0.2, stratify = y)    # labels of x_train is stored in y_train ----> same row # result

Training the model : Logistic Regression Model

In [42]:
model = LogisticRegression()


In [43]:
model.fit(x_train, y_train)

LogisticRegression()

Model Evaluation

using accuracy score 

In [45]:
# accuracy score of the traing data
x_train_prediction = model.predict(x_train)   # produces 0, 1 as output
training_data_accuracy = accuracy_score(x_train_prediction,y_train)   # Comparing the producted output to the actual output i.e, label

In [47]:
print("Accuracy score of the training data is :", training_data_accuracy)

Accuracy score of the training data is : 0.9412259615384615


In [49]:
# accuracy Score on test data 
x_test_prediction = model.predict(x_test)
test_training_data_accuracy = accuracy_score(x_test_prediction,y_test)
print("Accuracy score on test data is:", test_training_data_accuracy)

Accuracy score on test data is: 0.9257211538461538


So here above the Accuracy score i got is 92% which is good 

In [50]:
#

Building a predictive Model

In [58]:
x_new = x_test[500]
prediction = model.predict(x_new)
print(prediction)
if (prediction[0] == 0):
  print("The news is Real")
else:
  print("The news is Fake")

[0]
The news is Real


In [59]:
print(y_test[500])

0
